In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://172.17.0.2:27017/")

In [3]:
db = client.ecommerce
orders = db.orders

In [4]:
orders.insert_many([
    {
        "order_id": "001",
        "customer": {
        "name": "John Doe",
        "email": "john.doe@example.com"
        },
        "items": [
        {"product": "Laptop", "quantity": 1, "price": 1000},
        {"product": "Mouse", "quantity": 3, "price": 50}
        ],
        "order_date": "2024-01-15",
        "status": "shipped",
        "total": 1150
    },
    {
        "order_id": "002",
        "customer": {
        "name": "Emily Carter",
        "email": "emily.carter@example.com"
        },
        "items": [
        {"product": "Desk Chair", "quantity": 1, "price": 150},
        {"product": "Table", "quantity": 1, "price": 350}
        ],
        "order_date": "2024-02-13",
        "status": "shipped",
        "total": 500
    },
    {
        "order_id": "003",
        "customer": {
        "name": "John Doe",
        "email": "john.doe@example.com"
        },
        "items": [
        {"product": "Speaker", "quantity": 1, "price": 150},
        ],
        "order_date": "2024-02-11",
        "status": "shipped",
        "total": 150
    },
    {
        "order_id": "004",
        "customer": {
        "name": "Olivia Davis",
        "email": "olivia.davis@example.com"
        },
        "items": [
        {"product": "TV Unit", "quantity": 1, "price": 670},
        ],
        "order_date": "2024-05-01",
        "status": "shipped",
        "total": 670
    },
    {
        "order_id": "005",
        "customer": {
            "name": "Michael Smith",
            "email": "michael.smith@example.com"
        },
        "items": [
            {"product": "Headphones", "quantity": 1, "price": 200},
            {"product": "Webcam", "quantity": 1, "price": 100}
        ],
        "order_date": "2024-03-10",
        "status": "shipped",
        "total": 300
    },
    {
        "order_id": "006",
        "customer": {
            "name": "Sophia Johnson",
            "email": "sophia.johnson@example.com"
        },
        "items": [
            {"product": "Smartphone", "quantity": 1, "price": 800},
            {"product": "Phone Case", "quantity": 2, "price": 25}
        ],
        "order_date": "2024-03-15",
        "status": "shipped",
        "total": 850
    },
    {
        "order_id": "007",
        "customer": {
            "name": "John Doe",
            "email": "john.doe@example.com"
        },
        "items": [
            {"product": "Gaming Console", "quantity": 1, "price": 400},
            {"product": "Game Disk", "quantity": 2, "price": 60}
        ],
        "order_date": "2024-04-20",
        "status": "shipped",
        "total": 520
    },
    {
        "order_id": "008",
        "customer": {
            "name": "Ava Martinez",
            "email": "ava.martinez@example.com"
        },
        "items": [
            {"product": "Tablet", "quantity": 1, "price": 300},
            {"product": "Laptop", "quantity": 1, "price": 1000}
        ],
        "order_date": "2024-06-05",
        "status": "shipped",
        "total": 1300
    },
    {
        "order_id": "009",
        "customer": {
            "name": "John Doe",
            "email": "john.doe@example.com"
        },
        "items": [
            {"product": "Printer", "quantity": 1, "price": 250},
            {"product": "Ink Cartridge", "quantity": 2, "price": 30}
        ],
        "order_date": "2024-06-10",
        "status": "shipped",
        "total": 310
    }
])

InsertManyResult([ObjectId('690f23e785c9a5151834c01c'), ObjectId('690f23e785c9a5151834c01d'), ObjectId('690f23e785c9a5151834c01e'), ObjectId('690f23e785c9a5151834c01f'), ObjectId('690f23e785c9a5151834c020'), ObjectId('690f23e785c9a5151834c021'), ObjectId('690f23e785c9a5151834c022'), ObjectId('690f23e785c9a5151834c023'), ObjectId('690f23e785c9a5151834c024')], acknowledged=True)

In [5]:
# Filtering orders based on customer name and price
#Projecting only order id and total
filtered_orders = orders.find(
    {'customer.name': 'John Doe', "total": {"$gt": 500}},
    {"order_id": 1, "total": 1}
)

for order in filtered_orders:
    print(f"\nOrder ID: {order['order_id']} - Total {order['total']}")



Order ID: 001 - Total 1150

Order ID: 007 - Total 520


In [6]:
# Finding all orders
# Projecting order id, date, and total
# Sorting by order date

sorted_orders = orders.find({},
    {'order_id': 1, 'order_date': 1, 'total': 1}
).sort('order_date', -1)

for order in sorted_orders:
    print(f"\nOrder ID: {order['order_id']} - Date: {order['order_date']} - Total {order['total']}")



Order ID: 009 - Date: 2024-06-10 - Total 310

Order ID: 008 - Date: 2024-06-05 - Total 1300

Order ID: 004 - Date: 2024-05-01 - Total 670

Order ID: 007 - Date: 2024-04-20 - Total 520

Order ID: 006 - Date: 2024-03-15 - Total 850

Order ID: 005 - Date: 2024-03-10 - Total 300

Order ID: 002 - Date: 2024-02-13 - Total 500

Order ID: 003 - Date: 2024-02-11 - Total 150

Order ID: 001 - Date: 2024-01-15 - Total 1150


In [7]:
# Aggregation Pipeline for retrieving total sales per product
# Unwinding the items array so we can group by product name
# Adjusting the total field after unwinding
# Grouping based on product name and summing the total to get the total sales per product

total_sales_per_product = orders.aggregate([
    {
        '$unwind': {
            'path': '$items', 
            'includeArrayIndex': 'string', 
            'preserveNullAndEmptyArrays': True
        }
    }, {
        '$addFields': {
            'total': {
                '$multiply': [
                    '$items.price', '$items.quantity'
                ]
            }
        }
    }, {
        '$group': {
            '_id': '$items.product', 
            'total_sales': {
                '$sum': '$total'
            }
        }
    }
])

In [8]:
for product in total_sales_per_product:
    print(f"\nProduct: {product['_id']} - Total Sales: {product['total_sales']}")


Product: Headphones - Total Sales: 200

Product: Smartphone - Total Sales: 800

Product: Webcam - Total Sales: 100

Product: Game Disk - Total Sales: 120

Product: Tablet - Total Sales: 300

Product: Phone Case - Total Sales: 50

Product: Table - Total Sales: 350

Product: Ink Cartridge - Total Sales: 60

Product: Speaker - Total Sales: 150

Product: TV Unit - Total Sales: 670

Product: Gaming Console - Total Sales: 400

Product: Laptop - Total Sales: 2000

Product: Mouse - Total Sales: 150

Product: Printer - Total Sales: 250

Product: Desk Chair - Total Sales: 150


In [9]:
# Aggregation pipeline to retrieve average order value per customer
# Grouping by customer name and averaging the total to get the average order value
average_order_value_per_customer = orders.aggregate([
    {
        '$group': {
            '_id': '$customer.name', 
            'averageOrderValue': {
                '$avg': '$total'
            }
        }
    }
])

for customer in average_order_value_per_customer:
    print(f"Customer: {customer['_id']} - Average Order: {customer['averageOrderValue']}")

Customer: Michael Smith - Average Order: 300.0
Customer: Ava Martinez - Average Order: 1300.0
Customer: Olivia Davis - Average Order: 670.0
Customer: Emily Carter - Average Order: 500.0
Customer: John Doe - Average Order: 532.5
Customer: Sophia Johnson - Average Order: 850.0


In [10]:
# Aggregation pipeline to retrieve top products
# Unwinding the items array to group by product name
# Grouping by prodcut name and summing the quantity to get the quantity sold per product
# Sorting by quantity sold and limiting to top 5

top_products = orders.aggregate([
    {
        '$unwind': {
            'path': '$items', 
            'includeArrayIndex': 'string', 
            'preserveNullAndEmptyArrays': True
        }
    }, {
        '$group': {
            '_id': '$items.product', 
            'quantitySold': {
                '$sum': '$items.quantity'
            }
        }
    }, {
        '$sort': {
            'quantitySold': -1
        }
    }, {
        '$limit': 5
    }
])

for product in top_products:
    print(f"Product: {product['_id']} - Quantity Sold: {product['quantitySold']}")

Product: Mouse - Quantity Sold: 3
Product: Ink Cartridge - Quantity Sold: 2
Product: Game Disk - Quantity Sold: 2
Product: Laptop - Quantity Sold: 2
Product: Phone Case - Quantity Sold: 2
